# 📌 리뷰 데이터 저장

In [19]:
import dotenv
import os
import requests
import json
import pymongo
import pandas as pd

In [ ]:
# 애니메이션별 리뷰 목록 불러오기
def get_reviews(data):
    
    # 조회할 애니메이션 번호
    animation = data["id"]

    laftel_api = "https://laftel.net/api/reviews/v2/list/?item_id=" + str(animation)

    # 헤더에 지정해줘야 조회 가능
    header = {"laftel": "TeJava"}

    # 다음 데이터 불러올 api
    next = ""

    # 받아온 리뷰 데이터 저장
    review_list = []

    # next가 없을 때까지 api 호출해서 결과 저장
    while(True):
      response = requests.get(url=laftel_api, headers=header)
      json = response.json()
      results = json["results"]

      # 리뷰 내용만 가져오기
      for result in results:
        result["animation"] = animation
        review_list.append(result)

      next = json["next"]

      # 다음 가져올 데이터가 있다면 api 호출
      if (next is not None):
        laftel_api = next
      else:
        break


    # 애니메이션 번호-리뷰리스트 저장
    for r in review_list:
        dbcol_review.insert_one(r)
        
    # 완료된 애니메이션 -> 체크
    dbcol_ani.update_one({"id": animation}, {"$set": {"check": 1}})
    print(data["name"] + "- 완료")
    
# 환경변수 불러오기
dotenv.load_dotenv(dotenv.find_dotenv())
USER = os.environ["MONGODB_USER"] # MongoDB user
PASSWORD = os.environ["MONGODB_PW"] # MongoDB password
PORT = int(os.environ["MONGODB_PORT"]) # MongoDB port

# DB 연결
# conn = pymongo.MongoClient()
conn = pymongo.MongoClient("mongodb://" + USER + ":" + PASSWORD + "@j7e104.p.ssafy.io", PORT)
db = conn.test
dbcol_ani = db.animation
dbcol_review = db.review

# 체크 안한 애니메이션만 불러오기
df = pd.DataFrame(list(dbcol_ani.find({"$or": [{"check": {"$exists": False}}, {"check": {"$eq": 0}}]})))

df.apply(get_reviews, axis=1)
